In [13]:
from sys import path
path.append('../..')
from getData import getProducts

def normalizeData():
    products = getProducts()
    justProducts = []
    categories = list(products.keys())
    category_to_id = {category: idx for idx, category in enumerate(categories)}
    
    for category in products:
        category_id = category_to_id[category]
        for name, price in products[category]:
            justProducts.append((category, category_id, name, price))
    
    normalizedProducts = []
    for i in range(len(justProducts)):
        normalizedProducts.append({
            "product_id": i,                      
            "category_name": justProducts[i][0],     
            "category_id": justProducts[i][1],     
            "product_name": justProducts[i][2],     
            "product_price": justProducts[i][3]      
        })
    return normalizedProducts

In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp 

products = normalizeData()
# Reformat the products
reformattedProducts = {
    "category_name": [],
    "product_name": [],
    "product_price": []
}
for product in products:
    reformattedProducts["category_name"].append(product["category_name"])
    reformattedProducts["product_name"].append(product["product_name"])
    reformattedProducts["product_price"].append(product["product_price"])

# Create Data frame
df = pd.DataFrame(data=reformattedProducts)
# Create and configure encoder
ohe = pp.OneHotEncoder(handle_unknown = "ignore", sparse_output = False).set_output(transform = "pandas")
# Use encoder by fiting and transforming the data
oheTransformed = ohe.fit_transform(df[["category_name"]])
# Put both Data Frames together and delete the original category_name column
df = pd.concat([df, oheTransformed], axis=1).drop(columns=["category_name"])
df.head(5)


,product_name,product_price,category_name_accessories,category_name_baby,category_name_dresses,category_name_jackets,category_name_jeans,category_name_other,category_name_pants,category_name_sets,category_name_shirts,category_name_skirts,category_name_sleep,category_name_sweaters,category_name_swimwear,category_name_underwear,category_name_vests
0,Baby-Jacke mit Kapuze - geblümt,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Blümchen - 3-in-1-Baby-Jacke mit Kapuze,29.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Baby-Jacke mit Kapuze - wasserabweisend,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Blümchen - Baby-Jacke mit Kapuze,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baby-Steppjacke mit Kapuze - geblümt,19.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(df["product_name"])
product_names_encoded = vectorizer.fit_transform(df["product_name"])
product_names_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1520 stored elements and shape (423, 100)>

In [16]:
product_names_df = pd.DataFrame(
    product_names_encoded.toarray(),
    columns=vectorizer.get_feature_names_out()
)
df = pd.concat([df, product_names_df], axis=1).drop(columns=["product_name"])
df.head(5)

,product_price,category_name_accessories,category_name_baby,category_name_dresses,category_name_jackets,category_name_jeans,category_name_other,category_name_pants,category_name_sets,category_name_shirts,...,sweatjacke,sweatshirt,tapered,teilig,und,waist,wasserabweisend,way,weste,wide
0,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,29.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.528307,0.0,0.0,0.0
3,22.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,19.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
